# 基于DNN的花卉识别练习

导入需要的包

In [1]:
# “skimage” python 图像处理模块，如果用的conda的虚拟环境，需要用conda install scikit-image 进行安装
# glob 文件路径查找模块
# cv2图像处理包，比skimage要快
import cv2
from skimage import io,transform
import glob 
import os
import tensorflow as tf
import numpy as np

数据集路径---根据自己的路径进行改写

In [2]:
path='/home/liuyanfang/lyf/Codes/GAN/flower_photos/'

由于我们的数据集图片大小不一致，所以需要resize成统一大小 这里resize成100x100x3

In [3]:
w=100
h=100
c=3


读取数据集图片并添加标签,最后的形式是data 对应图片, label 是标签，roses 0,daisy 1,sunflowers 2,tulips 3,dandelion 4.

In [4]:
def read_img(path):
    imgs=[]
    labels=[]
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]
    for idx,i in enumerate(cate):
        for j in os.listdir(i):
            im = cv2.imread(i+'/'+j)
            img = cv2.resize(im, (w, h))
            #print('reading the images:%s'%(i+'/'+j))
            imgs.append(img)
            labels.append(idx)
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)
data,label=read_img(path)

In [4]:
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]#逐个打开文件夹，并提取图片
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):#用于在for循环中得到计数
        for im in glob.glob(folder+'/*.jpg'):#返回所有匹配的文件路径列表
            #print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
                  
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)#变成矩阵格式
data,label=read_img(path)

/home/yuyue/anaconda2/envs/tensorflow-gpu/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


将数据集打乱顺序，下阶段课程会讲其他方法

In [5]:
num_example=data.shape[0] # data.shape是(3029, 100, 100, 3)
arr=np.arange(num_example)# 创建等差数组 0，1，...,3028
np.random.shuffle(arr)# 打乱顺序
data=data[arr]
label=label[arr]
print(label)


[4 1 3 ... 3 4 1]


将标签onehot

In [6]:
def to_one_hot(labels):
    l = len(labels)
    res = np.zeros((l, 5), dtype=np.float32)
    for i in range(l):
        res[i][labels[i]] = 1.
    return res
label_oh = to_one_hot(label)


将所有数据集分为训练集80%、测试集20%

In [7]:
ratio=0.8
s=np.int(num_example*ratio)
x_train=data[:s]
y_train=label_oh[:s]
x_val=data[s:]
y_val=label_oh[s:]

生成batch

In [8]:
def gen_batch(dataset, labelset, batchsize):
    for i in range(dataset.shape[0]//batchsize):
        pos = i * batchsize
        x = dataset[pos:pos + batchsize]
        y = labelset[pos:pos + batchsize]
        yield x,y
    remain = np.shape(dataset)[0] % batchsize
    if remain != 0:
        x = dataset[-remain:]
        y = labelset[-remain:]
        yield x,y

构建网络

In [9]:
graph = tf.Graph()
with graph.as_default():
    X=tf.placeholder(tf.float32,shape=[None,w,h,c],name='X')
    y_=tf.placeholder(tf.int32,shape=[None,5],name='y_')
    keep_prob=tf.placeholder(tf.float32)

In [10]:
with graph.as_default():
    with tf.name_scope('hidden_layers'):
        conv1=tf.layers.conv2d(inputs=X,  filters=32,  kernel_size=[5, 5], padding="same", activation=tf.nn.relu,
          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    #第二个卷积层(50->25)
        conv2=tf.layers.conv2d( inputs=pool1, filters=64,  kernel_size=[5, 5], padding="same", activation=tf.nn.relu,
          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        pool2=tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    #第三个卷积层(25->12)
        conv3=tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu,
          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        pool3=tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

    #第四个卷积层(12->6)
        conv4=tf.layers.conv2d(  inputs=pool3, filters=128, kernel_size=[3, 3], padding="same",activation=tf.nn.relu,
          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        pool4=tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    

    #全连接层
    with tf.name_scope('hidden_layers'):
        print(pool4.get_shape())
        re1 = tf.reshape(pool4,shape=[-1, np.prod(pool4.get_shape()[1:])])
        dense1 = tf.layers.dense(inputs=re1,      units=1024,   activation=tf.nn.relu,
                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                          kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
        h_fc_drop1 = tf.nn.dropout(dense1, keep_prob)
        dense2= tf.layers.dense(inputs=h_fc_drop1,  units=512,  activation=tf.nn.relu,
                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                          kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
        h_fc_drop2 = tf.nn.dropout(dense2, keep_prob)
        logits= tf.layers.dense(inputs=h_fc_drop2,  units=5,   activation=None,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                            kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
    #---------------------------网络结束-----
        

(?, 6, 6, 128)


In [11]:
with graph.as_default():
    with tf.name_scope("cal_loss"):
        #loss = tf.losses.sparse_softmax_cross_entropy(labels=y_,logits=logits)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=logits))
    with tf.name_scope("train"):
        train_op=tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)
    with tf.name_scope("cal_accuracy"):
        true = tf.argmax(y_, axis=1)
        correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1),tf.int64), true)    
        acc= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    summary_loss = tf.summary.scalar('cost', loss)
    summary_acc = tf.summary.scalar('accuracy', acc)

训练和测试数据

In [12]:
epoch=200
batch_size=64
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tmp/visualization/',sess.graph)
    saver = tf.train.Saver()
    
    step = 0
    for epc in range(epoch):
        for x, y in gen_batch(x_train, y_train, batch_size):
            l, ac, _,s_m = sess.run([loss, acc, train_op, merged], feed_dict={X:x,y_:y,keep_prob:0.5})
            if step % 500 == 0:
                print("Step: {:>4}, Loss: {:.4f}, Acc: {:.4%}".format(step, l, ac))
            step += 1
            writer.add_summary(s_m, global_step=epc)
    print("Training finished.")
    l, ac, _ = sess.run([loss, acc, train_op],
                                       {X: x_val, y_: y_val,keep_prob:1})
    print("Testing Loss: {:.4f}, Testing Acc: {:.4%}".format(l, ac))

Step:    0, Loss: 1.6091, Acc: 23.4375%
Step:  500, Loss: 0.7481, Acc: 71.8750%
Step: 1000, Loss: 0.3975, Acc: 81.2500%
Step: 1500, Loss: 0.2343, Acc: 93.7500%
Step: 2000, Loss: 0.0552, Acc: 96.8750%
Step: 2500, Loss: 0.0370, Acc: 98.4375%
Step: 3000, Loss: 0.0064, Acc: 100.0000%
Step: 3500, Loss: 0.0317, Acc: 98.4375%
Step: 4000, Loss: 0.0246, Acc: 100.0000%
Step: 4500, Loss: 0.0536, Acc: 98.4375%
Step: 5000, Loss: 0.0085, Acc: 100.0000%
Step: 5500, Loss: 0.0012, Acc: 100.0000%
Step: 6000, Loss: 0.0004, Acc: 100.0000%
Step: 6500, Loss: 0.0002, Acc: 100.0000%
Step: 7000, Loss: 0.0001, Acc: 100.0000%
Step: 7500, Loss: 0.0002, Acc: 100.0000%
Step: 8000, Loss: 0.0028, Acc: 100.0000%
Step: 8500, Loss: 0.0014, Acc: 100.0000%
Step: 9000, Loss: 0.0016, Acc: 100.0000%
Training finished.
Testing Loss: 2.9787, Testing Acc: 69.7548%
